In [1]:
from lasertram import LaserTRAM, LaserCalc
import lasertram as lt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import mendeleev
def oxide_to_ppm(wt_percent, int_std):
    """
    convert concentration internal standard analyte oxide in weight percent to
    concentration ppm for a 1D series of data

    Parameters
    ----------
    wt_percent : array-like
        the oxide values to be converted to ppm
    int_std : string
        the internal standard used in the experiment (e.g., '29Si', '43Ca',
                                                      '47Ti')
        currently supported elements:

            - 'SiO2'
            - 'TiO2'
            - 'Al2O3'
            - 'Cr2O3'
            - 'MnO'
            - 'FeO'
            - 'K2O'
            - 'CaO'
            - 'Na2O'
            - 'NiO'
            - 'MgO'

    Returns
    -------
    ppm : array-like
        concentrations in ppm the same shape as the wt_percent input

    """

    el = [i for i in int_std if not i.isdigit()]

    if len(el) == 2:
        element = el[0] + el[1]

    else:
        element = el[0]

    oxides = [
        "SiO2",
        "TiO2",
        "Al2O3",
        "Cr2O3",
        "MnO",
        "FeO",
        "K2O",
        "CaO",
        "Na2O",
        "NiO",
        "MgO",
    ]

    for o in oxides:
        if element in o:
            oxide = o

    s = oxide.split("O")
    cat_subscript = s[0]
    an_subscript = s[1]

    cat_subscript = [i for i in cat_subscript if i.isdigit()]
    if cat_subscript:
        cat_subscript = int(cat_subscript[0])
    else:
        cat_subscript = 1

    an_subscript = [i for i in an_subscript if i.isdigit()]
    if an_subscript:
        an_subscript = int(an_subscript[0])
    else:
        an_subscript = 1

    ppm = 1e4 * (
        (wt_percent * mendeleev.element(element).atomic_weight * cat_subscript)
        / (
            mendeleev.element(element).atomic_weight
            + mendeleev.element("O").atomic_weight * an_subscript
        )
    )
    return ppm


In [2]:
data = pd.read_excel(r"C:\Users\jlubbers\OneDrive - DOI\Research\Coding\laserTRAM-DB\tests\spot_test_timestamp_raw_data.xlsx").set_index('SampleLabel')
data.head()

,timestamp,Time,7Li,24Mg,27Al,29Si,43Ca,48Ti,57Fe,88Sr,138Ba,139La,140Ce,153Eu,208Pb
SampleLabel,,,,,,,,,,,,,,,
BCR-2G_1,2021-03-01 22:08:14,13.24,500.010000,0.0,100.0004,46285.535670,100.0004,0.0000,600.014400,0.0,0.0,0.0,0.0,0.0,100.0004
BCR-2G_1,2021-03-01 22:08:14,163.60,700.019601,0.0,200.0016,43274.778818,0.0000,200.0016,900.032401,0.0,0.0,0.0,0.0,0.0,0.0000
BCR-2G_1,2021-03-01 22:08:14,313.96,600.014400,0.0,400.0064,46586.651171,300.0036,200.0016,1000.040002,0.0,0.0,0.0,0.0,0.0,200.0016
BCR-2G_1,2021-03-01 22:08:14,464.23,800.025601,0.0,100.0004,45683.326387,100.0004,100.0004,1400.078404,0.0,0.0,0.0,0.0,0.0,0.0000
BCR-2G_1,2021-03-01 22:08:14,614.50,600.014400,0.0,300.0036,46185.165445,200.0016,200.0016,700.019601,0.0,0.0,0.0,0.0,0.0,0.0000


In [3]:
samples = data.index.unique().dropna().tolist()

sample = samples[0]

bkgd_interval = (5,10)
keep_interval = (20,50)

spot = LaserTRAM(name = sample)
spot.get_data(data.loc[sample,:])
spot.assign_int_std("29Si")
spot.assign_intervals(bkgd_interval, keep_interval)
spot.get_bkgd_data()
spot.subtract_bkgd()
spot.get_detection_limits()
spot.subtract_bkgd()
spot.normalize_interval()
spot.make_output_report()

In [4]:
spot = LaserTRAM(name = sample)
lt.process_spot(
        spot,
        raw_data = data.loc[sample,:],
        bkgd = (5,10),
        keep = (20,50),
        internal_std = "29Si",
        despike = False,
        output_report = True
        )

In [5]:
bkgd_interval = (5,8)
keep_interval = (25,45)

int_std = "29Si"

my_spots = []
print("PROCESSING SPOTS\n")
for sample in tqdm(samples):
    spot = LaserTRAM(name = sample)
    lt.process_spot(
        spot,
        raw_data = data.loc[sample,:],
        bkgd = bkgd_interval,
        keep = keep_interval,
        internal_std = int_std,
        despike = False,
        output_report = True
        )
    my_spots.append(spot)

print("COMBINING OUTPUT REPORTS\n")
processed_df = pd.DataFrame()
for spot in tqdm(my_spots):
    processed_df = pd.concat([processed_df,spot.output_report])


PROCESSING SPOTS



  0%|          | 0/40 [00:00<?, ?it/s]

COMBINING OUTPUT REPORTS



  0%|          | 0/40 [00:00<?, ?it/s]

In [6]:
srm_data = pd.read_excel(r"C:\Users\jlubbers\OneDrive - DOI\Research\Coding\laserTRAM-DB\tests\laicpms_stds_tidy.xlsx")


In [7]:
concentrations = LaserCalc(name="experiment_test")
concentrations.get_SRM_comps(srm_data)
concentrations.get_data(processed_df)
concentrations.set_calibration_standard("BCR-2G")
concentrations.drift_check()
concentrations.get_calibration_std_ratios()
concentrations.set_internal_standard_concentrations(
    concentrations.data["Spot"],
    np.full(concentrations.data["Spot"].shape[0], 71.9),
    np.full(concentrations.data["Spot"].shape[0], 1),
)
concentrations.calculate_concentrations()
# concentrations.calculate_concentrations(SRM = True)
# concentrations.calculate_concentrations(SRM = False)


In [9]:
concentrations.unknown_concentrations / srm_data.set_index('Standard').loc['NIST-612',concentrations.elements].to_numpy()

,7Li,24Mg,27Al,29Si,43Ca,48Ti,57Fe,88Sr,138Ba,139La,140Ce,153Eu,208Pb
0,0.891205,0.866808,1.007761,0.999941,1.091779,10.522441,1.034384,1.124602,1.055383,1.157044,1.078661,1.161651,1.013653
1,0.906571,0.842899,1.002999,0.999941,1.087957,10.493219,1.092748,1.120797,1.053964,1.146525,1.077783,1.160084,1.001188
2,0.913992,0.882185,1.012040,0.999941,1.121267,10.925244,1.056840,1.149383,1.082358,1.194247,1.100061,1.181089,1.030895
3,0.861375,0.872835,1.023704,0.999941,1.102944,10.803386,1.000954,1.140821,1.082756,1.167138,1.079316,1.177700,1.028509
4,0.863712,0.883180,1.012253,0.999941,1.101301,10.738839,0.966697,1.132206,1.064677,1.139901,1.061408,1.167204,1.020851
